In [ ]:
!pip install mediapipe-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory_path = '/content/drive/MyDrive/Colab Notebooks/data'
directory_files = os.listdir(directory_path)
print(directory_files)
print(len(directory_files))

['train_val_stop', 'train_val_peace', 'train_val_peace_inverted', 'train_val_palm', 'train_val_rock', 'train_val_two_up', 'train_val_three2', 'train_val_two_up_inverted', 'train_val_three', 'train_val_stop_inverted', 'train_val_like', 'train_val_mute', 'train_val_one', 'train_val_four', 'train_val_call', 'train_val_ok', 'train_val_dislike', 'train_val_fist', 'train_val_zero']
19


In [ ]:
import shutil
def listdir(dir):
  return [f for f in os.listdir(dir) if not f.startswith('.')]
  
NONE_LABELS = ['train_val_peace', 'train_val_peace_inverted', 'train_val_stop', 'train_val_stop_inverted', 'train_val_three', 'train_val_three2', 'train_val_two_up', 'train_val_two_up_inverted']
IMGS_KEEP = 100
source_path = "/content/drive/MyDrive/Colab Notebooks/data"
result_path = "/content/drive/MyDrive/Colab Notebooks/gesture_recognizer_dataset"
if os.path.exists(result_path):
  shutil.rmtree(result_path)
os.mkdir(result_path)
for s_gest_name in listdir(source_path):
  s_gest_dir = os.path.join(source_path, s_gest_name)
  s_imgs = listdir(s_gest_dir)[:]
  if s_gest_name in NONE_LABELS:
    r_gest_dir = os.path.join(result_path, 'none')
  else:
    r_gest_dir = os.path.join(result_path, s_gest_name[len("train_val_"):])
  if not os.path.exists(r_gest_dir):
    os.mkdir(r_gest_dir)
  for img_name in s_imgs:
    shutil.copy(os.path.join(s_gest_dir, img_name), r_gest_dir)
dataset_path = result_path

In [ ]:
dataset_path

'/content/drive/MyDrive/Colab Notebooks/gesture_recognizer_dataset'

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

/content/drive/MyDrive/Colab Notebooks/gesture_recognizer_dataset
['none', 'palm', 'rock', 'like', 'mute', 'one', 'four', 'call', 'ok', 'dislike', 'fist', 'zero']


In [ ]:
# NUM_EXAMPLES = 5

# for label in labels:
#   label_dir = os.path.join(dataset_path, label)
#   example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
#   fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
#   for i in range(NUM_EXAMPLES):
#     axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
#     axs[i].get_xaxis().set_visible(False)
#     axs[i].get_yaxis().set_visible(False)
#   fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

# plt.show()

### Load the Dataset

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


### Train the Model

In [ ]:
hparams = gesture_recognizer.HParams(epochs=20,export_dir="exported_model")
model_options = gesture_recognizer.ModelOptions()
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams,model_options=model_options)

model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer)  [(None, 128)]            0         
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_o  (None, 12)               1548      
 ut (Dense)                                                      
                                                                 
Total params: 2,060
Trainable params: 1,804
Non-trainable p

### Evaluate the Model performance

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

157/157 [==============================] - 1s 2ms/step - loss: 0.5745 - categorical_accuracy: 0.8089
Test loss:0.5745160579681396, Test accuracy:0.808917224407196


### Run the Model

In [ ]:
# STEP 1:
import mediapipe as mp
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import ImageFormat
import numpy as np
from PIL import Image

In [ ]:
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

base_options = python.BaseOptions(model_asset_path='/content/drive/MyDrive/Colab Notebooks/gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

In [ ]:
import urllib

IMAGE_FILENAMES = ['thumbs_down.jpg', 'victory.jpg', 'thumbs_up.jpg', 'pointing_up.jpg']

for name in IMAGE_FILENAMES:
  url = f'https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/{name}'
  urllib.request.urlretrieve(url, name)

In [ ]:
for image_file_name in IMAGE_FILENAMES:
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_file_name)

  # STEP 4: Recognize gestures in the input image.
  recognition_result = recognizer.recognize(image)


images = []
results = []
for image_file_name in IMAGE_FILENAMES:
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_file_name)

  # STEP 4: Recognize gestures in the input image.
  recognition_result = recognizer.recognize(image)

  # STEP 5: Process the result. In this case, visualize it.
  images.append(image)
  top_gesture = recognition_result.gestures[0][0]
  hand_landmarks = recognition_result.hand_landmarks
  print(top_gesture.category_name,hand_landmarks)

dislike [[NormalizedLandmark(x=0.3775796592235565, y=0.4484468698501587, z=-7.658888421246957e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.45659855008125305, y=0.5906875729560852, z=-0.12274359911680222, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5610436201095581, y=0.7097854614257812, z=-0.21682056784629822, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6087026596069336, y=0.8151220083236694, z=-0.296338826417923, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6378511190414429, y=0.908829927444458, z=-0.35788753628730774, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.8904979825019836, y=0.6001406311988831, z=-0.19998541474342346, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.7954009175300598, y=0.6598494052886963, z=-0.3039839267730713, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6656880378723145, y=0.6365615129470825, z=-0.335548996925354, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6671821475028992, y=0.6085